In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import text
from jproperties import Properties
from datetime import timedelta
from app.wallet_processing import Wallet
from app.wallet_processing import Cashflow
from decimal import Decimal

### This notebook is a demo


In [2]:
w = Wallet(wallet_id = 1)


In [3]:
w.update_position(end_date='20250708') 

# this will update if there is any new transaction available in the database that weren't processed before, 
#based on the date (only works for new opeartions that are newer than the last date with position)

2025-07-08, D-1 position clonned
position updated!


In [4]:
w.get_position('20250530')

,date,wallet_id,brokerage_firm_id,asset,quantity,pu,value
0,2025-05-30,1,2,RECV3,300.00000000,14.19000000,4257.00000000
1,2025-05-30,1,1,POSI3,35.00000000,4.72000000,165.20000000


In [5]:
w.get_last_position()

,date,wallet_id,brokerage_firm_id,asset,quantity,pu,value
0,2025-07-08,1,2,RECV3,300.00000000,14.19000000,4257.00000000
1,2025-07-08,1,1,POSI3,35.00000000,4.72000000,165.20000000
2,2025-07-08,1,2,BBAS3,150.00000000,21.93000000,3289.50000000


In [6]:
w.get_operations('20250530') # retrieve the ops that are > than the passed date;

,id,date_op,wallet_id,movement,market,date_expiration,brokerage_firm_id,asset,quantity,pu,value
0,1,2025-05-30,1,Compra,Mercado Fracionário,2099-12-31,1,POSI3,17,4.72,80.24
1,2,2025-05-30,1,Compra,Mercado Fracionário,2099-12-31,1,POSI3,18,4.72,84.96
2,5,2025-06-06,1,Compra,Mercado à Vista,2099-12-31,2,BBAS3,100,21.75,2175.00
3,6,2025-06-05,1,Compra,Mercado Fracionário,2099-12-31,2,BBAS3,50,22.29,1114.50


### CASHFLOW CLASS

In [7]:
cashflow = Cashflow(wallet_id = 1)

In [8]:
# Example on how to insert a cashflow transfer 

#cashflow.cash_transfer(1011, 9, 101, 1, '20250520', 4257.0)
#cashflow.cash_transfer(1011, 9, 101, 1, '20250530', 3454.7)



In [ ]:
transaction_history = cashflow.get_transaction_history(start_date = '20230101' ,end_date = '20250605')

invested_value = transaction_history[transaction_history.origem_id.isna()].value.sum()
invested_value

Decimal('7711.70')

### PERFORMANCE ANALYSIS 

In [18]:
import yfinance as yf

In [19]:

df = w.get_last_position()
df.head(3)

,date,wallet_id,brokerage_firm_id,asset,quantity,pu,value
0,2025-07-08,1,2,RECV3,300.00000000,14.19000000,4257.00000000
1,2025-07-08,1,1,POSI3,35.00000000,4.72000000,165.20000000
2,2025-07-08,1,2,BBAS3,150.00000000,21.93000000,3289.50000000


In [ ]:
tickers = df.asset.values
tickers = " ".join(
    [ticker + '.SA' for ticker in tickers])

df_market_price = yf.Tickers(tickers).history('1mo')['Close']

###
df_market_price = df_market_price.reset_index().melt(id_vars = ['Date'])
df_market_price.columns = ['date', 'ticker', 'price_market']
df_market_price.ticker = [ticker.split('.')[0] for ticker in df_market_price.ticker]
df_market_price.loc[:, 'date'] = df_market_price.date.dt.date
df_market_price.set_index(['date', 'ticker'], inplace = True)

[*********************100%***********************]  3 of 3 completed


In [12]:
df_market_price.head(3)

,,price_market
date,ticker,
2025-06-09,BBAS3,21.730000
2025-06-10,BBAS3,21.540001
2025-06-11,BBAS3,21.400000


In [ ]:
df.date = pd.to_datetime(df.date)
df_pnl = df.join(df_market_price, on = ['date', 'asset'])

# conversion of dtypes
df_pnl.loc[:, 'price_market'] = df_pnl.price_market.apply(lambda x: Decimal(x))

# metrics calculation
df_pnl['market_value'] = df_pnl.price_market * df_pnl.quantity
df_pnl['PnL_%'] = (df_pnl.price_market/df_pnl.pu -1 ) * 100
df_pnl['PnL_R$'] = df_pnl['PnL_%']/100 * df_pnl.value

# round numbers; 
df_pnl = df_pnl.applymap(lambda x: round(x, 2) if isinstance(x, Decimal) else x)

In [16]:
df_pnl

,date,wallet_id,brokerage_firm_id,asset,quantity,pu,value,price_market,market_value,PnL_%,PnL_R$
0,2025-07-08,1,2,RECV3,300.00,14.19,4257.00,14.73,4419.00,3.81,162.00
1,2025-07-08,1,1,POSI3,35.00,4.72,165.20,4.92,172.20,4.24,7.00
2,2025-07-08,1,2,BBAS3,150.00,21.93,3289.50,22.00,3300.00,0.32,10.50


In [ ]:
print(f'Total return of the wallet -> {round((df_pnl.market_value.sum() / invested_value - 1) * 100, 4)}%')

Total return of the wallet -> 2.3276%
